In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import networkx as nx
import json
import itertools 

In [2]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data with baseline and noise
def generate_data(num_genes=100, num_samples=10, patients=5, controls=5):
    assert patients + controls == num_samples
    # Baseline expression levels for patients and controls
    baseline_patient = np.random.normal(loc=50, scale=10, size=num_genes)
    baseline_control = np.random.normal(loc=30, scale=10, size=num_genes)
    
    # Generate data for patients and controls with noise
    patient_data = np.array([np.random.negative_binomial(max(1, int(base)), 0.5, size=patients) for base in baseline_patient])
    control_data = np.array([np.random.negative_binomial(max(1, int(base)), 0.5, size=controls) for base in baseline_control])
    
    data = np.concatenate([patient_data, control_data], axis=1)
    columns = [f'p{i+1}' for i in range(patients)] + [f'c{i+1}' for i in range(controls)]
    df = pd.DataFrame(data, index=[f'g{i+1}' for i in range(num_genes)], columns=columns)
    return df

# Create datasets
datasets = {
    'USA': generate_data(),
    'CHN': generate_data(),
    'SA': generate_data(),
    'GER': generate_data()
}

In [3]:
datasets['USA']

,p1,p2,p3,p4,p5,c1,c2,c3,c4,c5
g1,61,65,61,101,49,21,16,14,12,22
g2,25,45,58,51,44,21,26,34,17,17
g3,38,71,42,57,66,28,28,39,17,19
g4,53,55,62,58,54,24,18,16,27,14
g5,43,64,59,32,45,23,30,18,17,40
...,...,...,...,...,...,...,...,...,...,...
g96,47,31,45,31,31,29,19,44,23,27
g97,71,45,51,56,57,19,27,14,13,27
g98,40,46,48,59,54,30,21,35,40,26
g99,55,59,48,54,42,29,20,23,36,40


In [4]:
# Normalize the data using log transformation (simulating RMA normalization)
def normalize_data(df):
    return np.log1p(df)

normalized_datasets = {k: normalize_data(v) for k, v in datasets.items()}

In [5]:
# Perform two-sample t-test
def perform_t_test(df):
    patients = df.columns[:5]
    controls = df.columns[5:]
    t_results = []
    for gene in df.index:
        t_stat, p_val = stats.ttest_ind(df.loc[gene, patients], df.loc[gene, controls])
        t_results.append((gene, t_stat, p_val))
    t_results_df = pd.DataFrame(t_results, columns=['Gene', 't_stat', 'p_val'])
    return t_results_df

t_test_results = {k: perform_t_test(v) for k, v in normalized_datasets.items()}

In [6]:
# Apply multiple testing correction (FDR)
def apply_fdr(df):
    p_vals = df['p_val'].values
    _, q_vals, _, _ = multipletests(p_vals, alpha=0.1, method='fdr_bh')
    df['q_val'] = q_vals
    return df

corrected_results = {k: apply_fdr(v) for k, v in t_test_results.items()}

In [7]:
corrected_results['USA']

,Gene,t_stat,p_val,q_val
0,g1,8.293142,0.000034,0.000425
1,g2,3.379468,0.009649,0.016404
2,g3,3.894228,0.004582,0.010182
3,g4,8.650333,0.000025,0.000413
4,g5,3.287387,0.011065,0.018140
...,...,...,...,...
95,g96,1.676895,0.132086,0.159140
96,g97,6.243460,0.000247,0.001025
97,g98,3.841077,0.004940,0.010611
98,g99,4.073805,0.003565,0.008897


In [8]:
# Filter genes based on p-value and q-value thresholds
def filter_genes(df, p_val_threshold=0.05, q_val_threshold=0.1):
    return df[(df['p_val'] < p_val_threshold) & (df['q_val'] < q_val_threshold)]

filtered_genes = {k: filter_genes(v) for k, v in corrected_results.items()}

for key, value in filtered_genes.items():
    print(f"{key}-{value.shape}")

USA-(73, 4)
CHN-(72, 4)
SA-(69, 4)
GER-(71, 4)


In [9]:
# Create graphs for each dataset
def create_graph(filtered_genes, normalized_data, threshold=0.7):
    graph = nx.Graph()
    genes = filtered_genes['Gene']
    for gene in genes:
        graph.add_node(gene)
    for i in range(len(genes)):
        for j in range(i + 1, len(genes)):
            gene1 = genes.iloc[i]
            gene2 = genes.iloc[j]
            # Adding edge with weight based on correlation of expression levels if above threshold
            weight = np.corrcoef(normalized_data.loc[gene1], normalized_data.loc[gene2])[0, 1]
            if abs(weight) > threshold:
                graph.add_edge(gene1, gene2, weight=weight)
    return graph

graph_dic = {k: create_graph(filtered_genes[k], normalized_datasets[k]) for k, v in filtered_genes.items()}

In [10]:
# Node strength analysis
def node_strength_analysis(graph):
    # Calculate Degree
    degree = nx.degree_centrality(graph)

    # Calculate Eccentricity
    eccentricity = nx.eccentricity(graph)

    # Calculate Closeness
    closeness = nx.closeness_centrality(graph)

    # Calculate Betweenness
    betweenness = nx.betweenness_centrality(graph)

    # Normalize values
    degree_max = max(degree.values())
    eccentricity_max = max(eccentricity.values())
    closeness_max = max(closeness.values())
    betweenness_max = max(betweenness.values())
    
    degree_normalized = {k: (v / degree_max if degree_max != 0 else 0) for k, v in degree.items()}
    eccentricity_normalized = {k: (v / eccentricity_max if eccentricity_max != 0 else 0) for k, v in eccentricity.items()}
    closeness_normalized = {k: (v / closeness_max if closeness_max != 0 else 0) for k, v in closeness.items()}
    betweenness_normalized = {k: (v / betweenness_max if betweenness_max != 0 else 0) for k, v in betweenness.items()}


    # Calculate Node Strength
    node_strength = {
        node: (degree_normalized[node] + eccentricity_normalized[node] + closeness_normalized[node] + betweenness_normalized[node]) / 4
        for node in graph.nodes()
    }

    # Convert results to DataFrame for better readability
    node_strength_df = pd.DataFrame({
        'Node': list(node_strength.keys()),
        'Degree': [degree[node] for node in node_strength.keys()],
        'Eccentricity': [eccentricity[node] for node in node_strength.keys()],
        'Closeness': [closeness[node] for node in node_strength.keys()],
        'Betweenness': [betweenness[node] for node in node_strength.keys()],
        'NodeStrength': list(node_strength.values())
    })

    return node_strength_df
    
# Perform node strength analysis for each graph
node_strength_results = {k: node_strength_analysis(v) for k, v in graph_dic.items()}

In [11]:
node_strength_results['USA']

,Node,Degree,Eccentricity,Closeness,Betweenness,NodeStrength
0,g1,0.888889,2,0.900000,0.029281,0.902841
1,g2,0.444444,2,0.642857,0.004125,0.497205
2,g3,0.500000,2,0.666667,0.004598,0.522846
3,g4,0.833333,2,0.857143,0.016896,0.770340
4,g5,0.375000,2,0.615385,0.001097,0.444975
...,...,...,...,...,...,...
68,g95,0.347222,2,0.605042,0.001026,0.433993
69,g97,0.638889,2,0.734694,0.005138,0.583759
70,g98,0.458333,2,0.648649,0.002088,0.485173
71,g99,0.569444,2,0.699029,0.004900,0.553126


In [12]:
# Calculate PCC
def calculate_pcc(gene1_data, gene2_data):
    return np.corrcoef(gene1_data, gene2_data)[0, 1]

# Example for one graph (USA)
usa_graph = graph_dic['USA']
normalized_data = normalized_datasets['USA']

# Initialize edge strength dictionary
edge_strengths = {}

for edge in usa_graph.edges():
    gene1, gene2 = edge
    pcc = calculate_pcc(normalized_data.loc[gene1], normalized_data.loc[gene2])
    edge_strengths[(gene1, gene2)] = {'PCC': pcc}

In [13]:
print("\n".join([f"{k}: {v}" for k, v in list(edge_strengths.items())[:10]]))

('g1', 'g2'): {'PCC': 0.7282491388045326}
('g1', 'g3'): {'PCC': 0.7653527213164605}
('g1', 'g4'): {'PCC': 0.8789570085087596}
('g1', 'g5'): {'PCC': 0.7507769564392135}
('g1', 'g7'): {'PCC': 0.89352960600505}
('g1', 'g10'): {'PCC': 0.7938443708009103}
('g1', 'g11'): {'PCC': 0.9128834471868577}
('g1', 'g12'): {'PCC': 0.8791171328799973}
('g1', 'g13'): {'PCC': 0.8601812558178221}
('g1', 'g18'): {'PCC': 0.9459181554886529}


In [14]:
# Calculate Gene Ontology Distance
def calculate_go_distance(go_annotations1, go_annotations2):
    go_union = go_annotations1.union(go_annotations2)
    go_intersection = go_annotations1.intersection(go_annotations2)
    go_symmetric_difference = go_annotations1.symmetric_difference(go_annotations2)
    if len(go_union) == 0:
        return 1
    return len(go_symmetric_difference) / (len(go_union) + len(go_intersection))

# Calculate Gene Ontology Distance
with open('/data/home/ysh980101/2405/result-rwr/GO-term.dic', 'r') as file:
    gene_go_annotations = json.load(file)
gene_go_annotations = {k: set(v) for k, v in gene_go_annotations.items()}
    
for edge in edge_strengths.keys():
    gene1, gene2 = edge
    go_distance = calculate_go_distance(gene_go_annotations[gene1], gene_go_annotations[gene2])
    edge_strengths[edge]['GO'] = go_distance

# Calculate Pathway Similarity    
def calculate_pathway_similarity(pathways1, pathways2):
    common_pathways = pathways1.intersection(pathways2)
    unique_pathways = pathways1.union(pathways2)
    if len(unique_pathways) == 0:
        return 0
    return len(common_pathways) / len(unique_pathways)

# Calculate Gene Ontology Distance
with open('/data/home/ysh980101/2405/result-rwr/gene-pathways.dic', 'r') as file:
    gene_pathways = json.load(file)
gene_pathways = {k: set(v) for k, v in gene_pathways.items()}

for edge in edge_strengths.keys():
    gene1, gene2 = edge
    pathway_similarity = calculate_pathway_similarity(gene_pathways[gene1], gene_pathways[gene2])
    edge_strengths[edge]['Pathway'] = pathway_similarity

In [15]:
print("\n".join([f"{k}: {v}" for k, v in list(edge_strengths.items())[:10]]))

('g1', 'g2'): {'PCC': 0.7282491388045326, 'GO': 1.0, 'Pathway': 0.20689655172413793}
('g1', 'g3'): {'PCC': 0.7653527213164605, 'GO': 1.0, 'Pathway': 0.13333333333333333}
('g1', 'g4'): {'PCC': 0.8789570085087596, 'GO': 1.0, 'Pathway': 0.03125}
('g1', 'g5'): {'PCC': 0.7507769564392135, 'GO': 1.0, 'Pathway': 0.0625}
('g1', 'g7'): {'PCC': 0.89352960600505, 'GO': 0.7142857142857143, 'Pathway': 0.19230769230769232}
('g1', 'g10'): {'PCC': 0.7938443708009103, 'GO': 0.7714285714285715, 'Pathway': 0.19230769230769232}
('g1', 'g11'): {'PCC': 0.9128834471868577, 'GO': 0.9333333333333333, 'Pathway': 0.13636363636363635}
('g1', 'g12'): {'PCC': 0.8791171328799973, 'GO': 1.0, 'Pathway': 0.13636363636363635}
('g1', 'g13'): {'PCC': 0.8601812558178221, 'GO': 1.0, 'Pathway': 0.23809523809523808}
('g1', 'g18'): {'PCC': 0.9459181554886529, 'GO': 1.0, 'Pathway': 0.18181818181818182}


In [16]:
# Normalize and Combine Features to Calculate Edge Strength
def normalize(values):
    min_val = min(values)
    max_val = max(values)
    return [(v - min_val) / (max_val - min_val) for v in values]

# Normalize the features
pcc_values = [v['PCC'] for v in edge_strengths.values()]
go_values = [v['GO'] for v in edge_strengths.values()]
pathway_values = [v['Pathway'] for v in edge_strengths.values()]

pcc_normalized = normalize(pcc_values)
go_normalized = normalize(go_values)
pathway_normalized = normalize(pathway_values)

for i, edge in enumerate(edge_strengths.keys()):
    edge_strengths[edge]['PCC_Norm'] = pcc_normalized[i]
    edge_strengths[edge]['GO_Norm'] = go_normalized[i]
    edge_strengths[edge]['Pathway_Norm'] = pathway_normalized[i]
    edge_strengths[edge]['EdgeStrength'] = (pcc_normalized[i] + go_normalized[i] + pathway_normalized[i]) / 3

In [17]:
print("\n".join([f"{k}: {v}" for k, v in list(edge_strengths.items())[:10]]))

('g1', 'g2'): {'PCC': 0.7282491388045326, 'GO': 1.0, 'Pathway': 0.20689655172413793, 'PCC_Norm': 0.8681905318695081, 'GO_Norm': 1.0, 'Pathway_Norm': 0.5172413793103448, 'EdgeStrength': 0.7951439703932843}
('g1', 'g3'): {'PCC': 0.7653527213164605, 'GO': 1.0, 'Pathway': 0.13333333333333333, 'PCC_Norm': 0.8873800558719053, 'GO_Norm': 1.0, 'Pathway_Norm': 0.3333333333333333, 'EdgeStrength': 0.7402377964017463}
('g1', 'g4'): {'PCC': 0.8789570085087596, 'GO': 1.0, 'Pathway': 0.03125, 'PCC_Norm': 0.9461348188425974, 'GO_Norm': 1.0, 'Pathway_Norm': 0.078125, 'EdgeStrength': 0.6747532729475325}
('g1', 'g5'): {'PCC': 0.7507769564392135, 'GO': 1.0, 'Pathway': 0.0625, 'PCC_Norm': 0.8798416466124945, 'GO_Norm': 1.0, 'Pathway_Norm': 0.15625, 'EdgeStrength': 0.6786972155374982}
('g1', 'g7'): {'PCC': 0.89352960600505, 'GO': 0.7142857142857143, 'Pathway': 0.19230769230769232, 'PCC_Norm': 0.9536715899708958, 'GO_Norm': 0.4285714285714286, 'Pathway_Norm': 0.4807692307692308, 'EdgeStrength': 0.62100408310

In [18]:
# Identify cliques of sizes 3 to 7
cliques = [clique for size in range(3, 8) for clique in nx.find_cliques(usa_graph) if len(clique) == size]

# Calculate clique strength
#def calculate_clique_strength(clique, node_strength, edge_strengths):
#    node_strength_sum = sum(node_strength[node] for node in clique)
#    edge_strength_sum = sum(edge_strengths[edge]['EdgeStrength'] for edge in combinations(clique, 2) if edge in edge_strengths)
#    return node_strength_sum + edge_strength_sum

# Calculate clique strength
def calculate_clique_strength(clique, node_strength, edge_strengths):
    node_strength_sum = sum(node_strength[node] for node in clique)
    edge_strength_sum = 0
    for i in range(len(clique)):
        for j in range(i + 1, len(clique)):
            edge = (clique[i], clique[j])
            if edge in edge_strengths:
                edge_strength_sum += edge_strengths[edge]['EdgeStrength']
            else:
                # Also consider reverse order of edge
                edge = (clique[j], clique[i])
                if edge in edge_strengths:
                    edge_strength_sum += edge_strengths[edge]['EdgeStrength']
    return node_strength_sum + edge_strength_sum

# Node strength (previously calculated)
node_strength = node_strength_results['USA'].set_index('Node')['NodeStrength'].to_dict()

# Calculate strength for each clique
clique_strengths = {tuple(clique): calculate_clique_strength(clique, node_strength, edge_strengths) for clique in cliques}

In [19]:
print("\n".join([f"{k}: {v}" for k, v in list(edge_strengths.items())[:10]]))

('g1', 'g2'): {'PCC': 0.7282491388045326, 'GO': 1.0, 'Pathway': 0.20689655172413793, 'PCC_Norm': 0.8681905318695081, 'GO_Norm': 1.0, 'Pathway_Norm': 0.5172413793103448, 'EdgeStrength': 0.7951439703932843}
('g1', 'g3'): {'PCC': 0.7653527213164605, 'GO': 1.0, 'Pathway': 0.13333333333333333, 'PCC_Norm': 0.8873800558719053, 'GO_Norm': 1.0, 'Pathway_Norm': 0.3333333333333333, 'EdgeStrength': 0.7402377964017463}
('g1', 'g4'): {'PCC': 0.8789570085087596, 'GO': 1.0, 'Pathway': 0.03125, 'PCC_Norm': 0.9461348188425974, 'GO_Norm': 1.0, 'Pathway_Norm': 0.078125, 'EdgeStrength': 0.6747532729475325}
('g1', 'g5'): {'PCC': 0.7507769564392135, 'GO': 1.0, 'Pathway': 0.0625, 'PCC_Norm': 0.8798416466124945, 'GO_Norm': 1.0, 'Pathway_Norm': 0.15625, 'EdgeStrength': 0.6786972155374982}
('g1', 'g7'): {'PCC': 0.89352960600505, 'GO': 0.7142857142857143, 'Pathway': 0.19230769230769232, 'PCC_Norm': 0.9536715899708958, 'GO_Norm': 0.4285714285714286, 'Pathway_Norm': 0.4807692307692308, 'EdgeStrength': 0.62100408310

In [20]:
# Clique Connectivity Profile Algorithm (CCP)
def calculate_clique_connectivity(clique1, clique2, clique_strengths):
    common_nodes = set(clique1).intersection(clique2)
    if len(common_nodes) > 0:
        return (clique_strengths[tuple(clique1)] + clique_strengths[tuple(clique2)]) / 2
    return 0

ccp = []
for i, clique1 in enumerate(cliques):
    for j, clique2 in enumerate(cliques):
        if i != j:
            connectivity_score = calculate_clique_connectivity(clique1, clique2, clique_strengths)
            if connectivity_score > 0:
                ccp.append((clique1, clique2, connectivity_score))

ccp_df = pd.DataFrame(ccp, columns=['Clique1', 'Clique2', 'ConnectivityScore'])

In [21]:
ccp_df

,Clique1,Clique2,ConnectivityScore
0,"[g34, g89, g61, g85]","[g89, g61, g59, g35, g73]",7.942419
1,"[g34, g89, g61, g85]","[g89, g61, g59, g35, g85]",8.030250
2,"[g89, g61, g59, g35, g73]","[g34, g89, g61, g85]",7.942419
3,"[g89, g61, g59, g35, g73]","[g89, g61, g59, g35, g85]",9.854560
4,"[g89, g61, g59, g35, g85]","[g34, g89, g61, g85]",8.030250
5,"[g89, g61, g59, g35, g85]","[g89, g61, g59, g35, g73]",9.854560
6,"[g29, g71, g77, g90, g1, g55]","[g29, g71, g77, g90, g1, g100, g3]",15.912614
7,"[g29, g71, g77, g90, g1, g55]","[g29, g71, g77, g90, g1, g100, g60]",15.829798
8,"[g29, g71, g77, g90, g1, g55]","[g29, g71, g77, g90, g1, g2, g3]",15.949575
9,"[g29, g71, g77, g90, g1, g55]","[g29, g71, g77, g90, g1, g2, g60]",15.808830


In [22]:
# Statistical Evaluation of Cliques Using Gene Enrichment Analysis
from scipy.stats import hypergeom

def gene_enrichment_analysis(clique, all_genes, go_annotations):
    M = len(all_genes)  # Total number of genes
    n = len(go_annotations)  # Total number of genes with the GO term
    N = len(clique)  # Number of genes in the clique
    x = sum(1 for gene in clique if gene in go_annotations)  # Number of genes in the clique with the GO term
    return hypergeom.sf(x - 1, M, n, N)

# GO term annotations 
with open('/data/home/ysh980101/2405/result-rwr/go_term_annotations.dic', 'r') as file:
    go_term_annotations = json.load(file)
go_term_annotations = {k: set(v) for k, v in go_term_annotations.items()} 

enrichment_results = []
for clique in cliques:
    for go_term, genes in go_term_annotations.items():
        p_value = gene_enrichment_analysis(clique, normalized_data.index, genes)
        if p_value < 0.05:
            enrichment_results.append((clique, go_term, p_value))

enrichment_df = pd.DataFrame(enrichment_results, columns=['Clique', 'GOTerm', 'p_value'])

In [23]:
enrichment_df

,Clique,GOTerm,p_value
0,"[g34, g89, g61, g85]",GO:1852264,0.020976
1,"[g34, g89, g61, g85]",GO:5446912,0.032499
2,"[g34, g89, g61, g85]",GO:9548432,0.048769
3,"[g34, g89, g61, g85]",GO:2876932,0.041938
4,"[g89, g61, g59, g35, g73]",GO:3396987,0.039441
5,"[g89, g61, g59, g35, g73]",GO:5672073,0.039441
6,"[g89, g61, g59, g35, g73]",GO:7377459,0.016228
7,"[g89, g61, g59, g35, g73]",GO:1971823,0.049517
8,"[g89, g61, g59, g35, g73]",GO:2166941,0.033442
9,"[g89, g61, g59, g35, g85]",GO:2724590,0.023156
